## Regression

In [18]:
from __future__ import absolute_import,division,print_function,unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

## Loading data in dataframe

In [19]:
dftrain = pd.read_csv('./data/train.csv')
dfeval  = pd.read_csv('./data/eval.csv')
# print(dftrain.head())
# print(dfeval.head())

# Popped out the survived col from the datasets
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived') 

# print(y_train.head())


In [3]:
dftrain.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [ ]:
numeric_col = ['age','fare']
cat_col = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck','embark_town', 'alone']
feature_col = []

for feature in cat_col:
    voc = dftrain[feature].unique()
    feature_col.append(tf.feature_column.categorical_column_with_vocabulary_list(feature,voc))
for feature in numeric_col:
    feature_col.append(tf.feature_column.numeric_column(feature, dtype=tf.float32))

print(feature_col)

## Input function
The TensorFlow model we are going to use requires that the data we pass it comes in as a ```tf.data.Dataset``` object. This means we must create a *input function* that can convert our current pandas dataframe into that object

In [21]:
def make_input_fun(data_f,label_f,nepo=10,batch_size=32,shuffle=True):
    def input_fun():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_f),label_f))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(nepo)
        return ds
    return input_fun

train_input = make_input_fun(dftrain,y_train)
eval_data = make_input_fun(dfeval,y_eval,nepo=1,shuffle=False)

## Now we need to create a model 

In [26]:
esti = tf.estimator.LinearClassifier(feature_columns=feature_col)
clear_output()

In [31]:
esti.train(train_input)
result = esti.evaluate(eval_data)
clear_output()
print(result['accuracy'])

0.7462121


In [32]:
pred_dicts = list(esti.predict(eval_data))
clear_output()
print(dfeval.loc[3])
print(y_eval.loc[3])
print(pred_dicts[3]['probabilities'][1])

sex                        female
age                          55.0
n_siblings_spouses              0
parch                           0
fare                         16.0
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 3, dtype: object
1
0.8116084
